In [ ]:
from oneatlas import OneAtlasClient
import json

In [ ]:
# function to convert polygon gdf to bounding box list

In [ ]:
# Load the JSON file
with open('config.json', 'r') as file:
    config = json.load(file)

# Access the API key
api_key = config['api_key']

In [ ]:
client = OneAtlasClient(api_key=api_key)

results = client.search({
    "cloudCover": "[0,30]",
    "incidenceAngle": "[0,40]",
    "processingLevel": "SENSOR",
    "relation": "intersects",
    "bbox":"-7.57216793459,49.959999905,1.68153079591,58.6350001085"
})
result = results['features'][0]
client.download_quicklook_to_file(result, "quicklook.jpg")

In [ ]:
results['features'][0]